In [1]:
import pandas as pd
import numpy as np
df = pd.read_csv('../Data/IMDB Dataset.csv')


In [2]:
df

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 50000 entries, 0 to 49999
Data columns (total 2 columns):
 #   Column     Non-Null Count  Dtype 
---  ------     --------------  ----- 
 0   review     50000 non-null  object
 1   sentiment  50000 non-null  object
dtypes: object(2)
memory usage: 781.4+ KB


In [4]:
df.head(1)

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive


In [5]:
#feature mapping of the sentiment 
feature_map = {'positive':1,'negative':0}
df['sentiment-map'] = df['sentiment'].map(feature_map)

In [6]:
df

,review,sentiment,sentiment-map
0,One of the other reviewers has mentioned that ...,positive,1
1,A wonderful little production. <br /><br />The...,positive,1
2,I thought this was a wonderful way to spend ti...,positive,1
3,Basically there's a family where a little boy ...,negative,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive,1
...,...,...,...
49995,I thought this movie did a down right good job...,positive,1
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative,0
49997,I am a Catholic taught in parochial elementary...,negative,0
49998,I'm going to have to disagree with the previou...,negative,0


In [7]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, Flatten
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import CountVectorizer

In [9]:
df['review'][:1000]

0      One of the other reviewers has mentioned that ...
1      A wonderful little production. <br /><br />The...
2      I thought this was a wonderful way to spend ti...
3      Basically there's a family where a little boy ...
4      Petter Mattei's "Love in the Time of Money" is...
                             ...                        
995    Nothing is sacred. Just ask Ernie Fosselius. T...
996    I hated it. I hate self-aware pretentious inan...
997    I usually try to be professional and construct...
998    If you like me is going to see this in a film ...
999    This is like a zoology textbook, given that it...
Name: review, Length: 1000, dtype: object

In [10]:
X = df['review'][:1001]
y = df['sentiment-map'][:1001]


In [11]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [12]:
vectorizer = CountVectorizer(stop_words='english')

In [13]:
#using transform on test data to avoid data leakage
X_train_vec = vectorizer.fit_transform(X_train)
X_test_vec = vectorizer.transform(X_test) 

In [14]:

model = Sequential()

In [15]:
X_train_vec.shape[1]

15592

In [16]:
model.add(Dense(64,activation='relu',input_shape=(X_train_vec.shape[1],)))
model.add(Dense(1,activation='sigmoid'))

In [17]:
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 64)                997952    
                                                                 
 dense_1 (Dense)             (None, 1)                 65        
                                                                 
Total params: 998017 (3.81 MB)
Trainable params: 998017 (3.81 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [19]:
X_train_dense = X_train_vec.toarray()

In [20]:
model.fit(X_train_dense, y_train, epochs=10, batch_size=32)

Epoch 1/10
25/25 [==============================] - 0s 3ms/step - loss: 0.6310 - accuracy: 0.6737
Epoch 2/10
25/25 [==============================] - 0s 3ms/step - loss: 0.2380 - accuracy: 0.9875
Epoch 3/10
25/25 [==============================] - 0s 3ms/step - loss: 0.0865 - accuracy: 0.9975
Epoch 4/10
25/25 [==============================] - 0s 3ms/step - loss: 0.0391 - accuracy: 0.9987
Epoch 5/10
25/25 [==============================] - 0s 3ms/step - loss: 0.0220 - accuracy: 1.0000
Epoch 6/10
25/25 [==============================] - 0s 3ms/step - loss: 0.0145 - accuracy: 1.0000
Epoch 7/10
25/25 [==============================] - 0s 4ms/step - loss: 0.0103 - accuracy: 1.0000
Epoch 8/10
25/25 [==============================] - 0s 5ms/step - loss: 0.0077 - accuracy: 1.0000
Epoch 9/10
25/25 [==============================] - 0s 5ms/step - loss: 0.0060 - accuracy: 1.0000
Epoch 10/10
25/25 [==============================] - 0s 4ms/step - loss: 0.0048 - accuracy: 1.0000


In [21]:
loss, accuracy = model.evaluate(X_test_vec, y_test)
print("Test Accuracy:", accuracy)

2024-02-13 10:19:55.376214: W tensorflow/core/framework/op_kernel.cc:1827] INVALID_ARGUMENT: TypeError: 'SparseTensor' object is not subscriptable
Traceback (most recent call last):

  File "/Users/uditrawat/anaconda3/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 268, in __call__
    return func(device, token, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^

  File "/Users/uditrawat/anaconda3/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 146, in __call__
    outputs = self._call(device, args)
              ^^^^^^^^^^^^^^^^^^^^^^^^

  File "/Users/uditrawat/anaconda3/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 153, in _call
    ret = self._func(*args)
          ^^^^^^^^^^^^^^^^^

  File "/Users/uditrawat/anaconda3/lib/python3.11/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^

  File "/Users/uditrawat/anaconda3/lib/pyth

InvalidArgumentError: Graph execution error:

Detected at node EagerPyFunc defined at (most recent call last):
<stack traces unavailable>
TypeError: 'SparseTensor' object is not subscriptable
Traceback (most recent call last):

  File "/Users/uditrawat/anaconda3/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 268, in __call__
    return func(device, token, args)
           ^^^^^^^^^^^^^^^^^^^^^^^^^

  File "/Users/uditrawat/anaconda3/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 146, in __call__
    outputs = self._call(device, args)
              ^^^^^^^^^^^^^^^^^^^^^^^^

  File "/Users/uditrawat/anaconda3/lib/python3.11/site-packages/tensorflow/python/ops/script_ops.py", line 153, in _call
    ret = self._func(*args)
          ^^^^^^^^^^^^^^^^^

  File "/Users/uditrawat/anaconda3/lib/python3.11/site-packages/tensorflow/python/autograph/impl/api.py", line 643, in wrapper
    return func(*args, **kwargs)
           ^^^^^^^^^^^^^^^^^^^^^